In [1]:
import pandas as pd
from pathlib import Path

def clean_master_df(df_raw: pd.DataFrame) -> pd.DataFrame:
    """
    Limpieza para master_dataset_final:
    - timestamp a datetime
    - elimina columnas completamente vacías
    - elimina columnas auxiliares de outliers (zscore, is_outlier)
    - imputa columnas orográficas numéricas con la mediana
    - elimina duplicados
    """
    df = df_raw.copy()

    # 1) timestamp
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    # 2) eliminar columnas totalmente vacías
    all_nan_cols = df.columns[df.isna().all()].tolist()
    df = df.drop(columns=all_nan_cols)

    # 3) quitar columnas de outliers (ya no las necesitamos)
    for col in ["zscore", "is_outlier"]:
        if col in df.columns:
            df = df.drop(columns=[col])

    # 4) imputar columnas numéricas orográficas
    cols_imputar = [
        "elevation_m",
        "slope_deg",
        "aspect_deg",
        "roughness_m",
        "gradient",
        "distance_to_river_m",
        "flood_risk",
        "landslide_risk",
    ]
    for col in cols_imputar:
        if col in df.columns:
            df[col] = df[col].fillna(df[col].median())

    # 5) quitar duplicados
    df = df.drop_duplicates()

    return df


def master_long_to_wide(df_clean: pd.DataFrame) -> pd.DataFrame:
    """
    Convierte master_dataset_final (formato largo con 'sensor' y 'value')
    a formato ancho: columnas TEMP, HUMEDAD, LLUVIA, VIENTO.
    """
    base_cols = [
        "timestamp",
        "station_id",
        "station_name",
        "lat",
        "lon",
        "source",
        "date",
        "hour",
        "year",
        "month",
        "day",
        "elevation_m",
        "slope_deg",
        "aspect_deg",
        "roughness_m",
        "gradient",
        "distance_to_river_m",
        "terrain_class",
        "flood_risk",
        "landslide_risk",
    ]

    # solo las que existan realmente
    base_cols = [c for c in base_cols if c in df_clean.columns]

    wide = (
        df_clean
        .pivot_table(
            index=base_cols,
            columns="sensor",
            values="value",
            aggfunc="mean"   # por si hay más de una medición por sensor
        )
        .reset_index()
    )

    wide.columns.name = None
    return wide


In [2]:
from pathlib import Path
import pandas as pd

# Detectar base del proyecto (si el notebook corre desde / o desde /notebooks)
BASE_DIR = Path(".").resolve()
csv_path = BASE_DIR / "data_clean" / "master" / "master_dataset_final.csv"

if not csv_path.exists():
    # Si estás parado dentro de /notebooks, subimos un nivel
    BASE_DIR = Path("..").resolve()
    csv_path = BASE_DIR / "data_clean" / "master" / "master_dataset_final.csv"

print("Leyendo:", csv_path)

df_raw = pd.read_csv(csv_path)
print(df_raw.shape)
display(df_raw.head())


Leyendo: C:\Python\Proyecto IA\Proyecto2--Python\data_clean\master\master_dataset_final.csv
(148, 35)


,timestamp,station_id,station_name,lat,lon,sensor,value,source,date,hour,...,is_outlier,elevation_m,slope_deg,aspect_deg,roughness_m,gradient,distance_to_river_m,terrain_class,flood_risk,landslide_risk
0,2025-12-03 12:00:00,140006,SE CHORRERA (140-006),8.9078,-79.7786,TEMP,27.5,IMHPA,2025-12-03,12:00:00,...,False,48.0,4.885025,71.565051,18.0,0.085467,313.741051,colinas,0.290701,0.155743
1,2025-12-03 12:00:00,140006,SE CHORRERA (140-006),8.9078,-79.7786,HUMEDAD,95.8,IMHPA,2025-12-03,12:00:00,...,False,48.0,NaN,NaN,NaN,NaN,313.741051,muy_montanoso,NaN,NaN
2,2025-12-10 08:45:00,102016,GOMEZ ARRIBA (102-016),8.5667,-82.7333,LLUVIA,7.5,IMHPA,2025-12-10,08:45:00,...,False,374.0,2.741431,41.185925,15.0,0.047884,727.154842,lomereo_suave,0.347326,0.136996
3,2025-12-10 09:00:00,108053,BOQUETE (108-053),8.7550,-82.4314,TEMP,25.6,IMHPA,2025-12-10,09:00:00,...,False,1102.0,NaN,NaN,NaN,NaN,343.491547,muy_montanoso,NaN,NaN
4,2025-12-10 09:00:00,108053,BOQUETE (108-053),8.7550,-82.4314,HUMEDAD,67.3,IMHPA,2025-12-10,09:00:00,...,False,1102.0,NaN,NaN,NaN,NaN,343.491547,muy_montanoso,NaN,NaN


In [3]:
# 1) limpiar
df_clean = clean_master_df(df_raw)
print("Después de limpieza:", df_clean.shape)
display(df_clean.head())

# 2) largo -> ancho
df_wide = master_long_to_wide(df_clean)
print("Formato ancho:", df_wide.shape)
display(df_wide.head())


Después de limpieza: (148, 22)


,timestamp,station_id,station_name,lat,lon,sensor,value,source,date,hour,...,day,elevation_m,slope_deg,aspect_deg,roughness_m,gradient,distance_to_river_m,terrain_class,flood_risk,landslide_risk
0,2025-12-03 12:00:00,140006,SE CHORRERA (140-006),8.9078,-79.7786,TEMP,27.5,IMHPA,2025-12-03,12:00:00,...,3.0,48.0,4.885025,71.565051,18.0,0.085467,313.741051,colinas,0.290701,0.155743
1,2025-12-03 12:00:00,140006,SE CHORRERA (140-006),8.9078,-79.7786,HUMEDAD,95.8,IMHPA,2025-12-03,12:00:00,...,3.0,48.0,2.079867,180.000000,8.0,0.036316,313.741051,muy_montanoso,0.302414,0.077784
2,2025-12-10 08:45:00,102016,GOMEZ ARRIBA (102-016),8.5667,-82.7333,LLUVIA,7.5,IMHPA,2025-12-10,08:45:00,...,10.0,374.0,2.741431,41.185925,15.0,0.047884,727.154842,lomereo_suave,0.347326,0.136996
3,2025-12-10 09:00:00,108053,BOQUETE (108-053),8.7550,-82.4314,TEMP,25.6,IMHPA,2025-12-10,09:00:00,...,10.0,1102.0,2.079867,180.000000,8.0,0.036316,343.491547,muy_montanoso,0.302414,0.077784
4,2025-12-10 09:00:00,108053,BOQUETE (108-053),8.7550,-82.4314,HUMEDAD,67.3,IMHPA,2025-12-10,09:00:00,...,10.0,1102.0,2.079867,180.000000,8.0,0.036316,343.491547,muy_montanoso,0.302414,0.077784


Formato ancho: (103, 24)


,timestamp,station_id,station_name,lat,lon,source,date,hour,year,month,...,roughness_m,gradient,distance_to_river_m,terrain_class,flood_risk,landslide_risk,HUMEDAD,LLUVIA,TEMP,VIENTO
0,2025-12-03 12:00:00,140006,SE CHORRERA (140-006),8.90780,-79.77860,IMHPA,2025-12-03,12:00:00,2025.0,12.0,...,8.0,0.036316,313.741051,muy_montanoso,0.302414,0.077784,95.8,NaN,NaN,NaN
1,2025-12-03 12:00:00,140006,SE CHORRERA (140-006),8.90780,-79.77860,IMHPA,2025-12-03,12:00:00,2025.0,12.0,...,18.0,0.085467,313.741051,colinas,0.290701,0.155743,NaN,NaN,27.5,NaN
2,2025-12-10 08:45:00,102016,GOMEZ ARRIBA (102-016),8.56670,-82.73330,IMHPA,2025-12-10,08:45:00,2025.0,12.0,...,15.0,0.047884,727.154842,lomereo_suave,0.347326,0.136996,NaN,7.5,NaN,NaN
3,2025-12-10 09:00:00,108053,BOQUETE (108-053),8.75500,-82.43140,IMHPA,2025-12-10,09:00:00,2025.0,12.0,...,8.0,0.036316,343.491547,muy_montanoso,0.302414,0.077784,67.3,NaN,25.6,NaN
4,2025-12-10 14:00:00,100136,BURICA CENTRO (100-136),8.38045,-82.87011,IMHPA,2025-12-10,14:00:00,2025.0,12.0,...,8.0,0.036316,116.362250,muy_montanoso,0.302414,0.077784,97.7,NaN,NaN,NaN


In [4]:
out_dir = BASE_DIR / "data_clean" / "master"
out_dir.mkdir(parents=True, exist_ok=True)

csv_clean = out_dir / "master_dataset_final_clean.csv"
parquet_clean = out_dir / "master_dataset_final_clean.parquet"

csv_wide = out_dir / "master_dataset_final_wide.csv"
parquet_wide = out_dir / "master_dataset_final_wide.parquet"

# guardar limpio (aún en formato largo)
df_clean.to_csv(csv_clean, index=False)
df_clean.to_parquet(parquet_clean, index=False)

# guardar ancho (columnas TEMP, HUMEDAD, LLUVIA, VIENTO)
df_wide.to_csv(csv_wide, index=False)
df_wide.to_parquet(parquet_wide, index=False)

print("Guardado:")
print(" -", csv_clean)
print(" -", parquet_clean)
print(" -", csv_wide)
print(" -", parquet_wide)


Guardado:
 - C:\Python\Proyecto IA\Proyecto2--Python\data_clean\master\master_dataset_final_clean.csv
 - C:\Python\Proyecto IA\Proyecto2--Python\data_clean\master\master_dataset_final_clean.parquet
 - C:\Python\Proyecto IA\Proyecto2--Python\data_clean\master\master_dataset_final_wide.csv
 - C:\Python\Proyecto IA\Proyecto2--Python\data_clean\master\master_dataset_final_wide.parquet
